# Probabilistic Data Structures

**Probabilistic Data Structures** represent a relatively new area of algorithms.
Notably, the mathematician [Philippe Flajolet](http://algo.inria.fr/flajolet/) is credited with early work, though a number of different people contributed in the examples shown here.

You may also hear the terms _approximation algorithms_ or _sketch algorithms_ used to describe the same body of work.
These are proving to be especially useful for analytics with large-scale data as well streaming data applications.

The main idea here is that these approaches provide *approximations with error bounds*. 
The amount of acceptable error is generally a trade-off for less system resources, such as memory or compute time.
Parameters adjust those error bounds, and also determine the resources required for a given application.

This notebook shows Python code for some the more well-known examples of probabilistic data structures, include:

|algorithm|usage|
|---|---|
|HyperLogLog|set cardinality|
|BloomFilter|set membership|
|MinHash|set similarity|
|Count-Min Sketch|frequency summaries|
|t-Digest|streaming quantiles|

### Data Sets

We'll need some interesting data to help illustrate how these kinds of algorithms work. 
Let's create a data set from the text of [Jabberwocky](http://www.jabberwocky.com/carroll/jabber/jabberwocky.html) by Lewis Carroll.

In [26]:
jabber_text = """
`Twas brillig, and the slithy toves
  Did gyre and gimble in the wabe:
All mimsy were the borogoves,
  And the mome raths outgrabe.

"Beware the Jabberwock, my son!
  The jaws that bite, the claws that catch!
Beware the Jubjub bird, and shun
  The frumious Bandersnatch!"
He took his vorpal sword in hand:
  Long time the manxome foe he sought --
So rested he by the Tumtum tree,
  And stood awhile in thought.
And, as in uffish thought he stood,
  The Jabberwock, with eyes of flame,
Came whiffling through the tulgey wood,
  And burbled as it came!
One, two! One, two! And through and through
  The vorpal blade went snicker-snack!
He left it dead, and with its head
  He went galumphing back.
"And, has thou slain the Jabberwock?
  Come to my arms, my beamish boy!
O frabjous day! Callooh! Callay!'
  He chortled in his joy.

`Twas brillig, and the slithy toves
  Did gyre and gimble in the wabe;
All mimsy were the borogoves,
  And the mome raths outgrabe.
"""

For comparison, we'll use text from another poem, [Daylight Saving](http://www.best-poems.net/dorothy_parker/daylight_saving.html) by Dorothy Parker.

In [28]:
parker_text = """
My answers are inadequate
To those demanding day and date
And ever set a tiny shock
Through strangers asking what's o'clock;
Whose days are spent in whittling rhyme-
What's time to her, or she to Time?
"""

Next, let's define a simple Python function to construct lists of words from these poems.

In [29]:
import re

def clean_words (text):
  return filter(lambda x: len(x) > 0, re.sub("[^A-Za-z]", " ", text).split(" "))

In [30]:
jabber_words = clean_words(jabber_text.lower())
print jabber_words

['twas', 'brillig', 'and', 'the', 'slithy', 'toves', 'did', 'gyre', 'and', 'gimble', 'in', 'the', 'wabe', 'all', 'mimsy', 'were', 'the', 'borogoves', 'and', 'the', 'mome', 'raths', 'outgrabe', 'beware', 'the', 'jabberwock', 'my', 'son', 'the', 'jaws', 'that', 'bite', 'the', 'claws', 'that', 'catch', 'beware', 'the', 'jubjub', 'bird', 'and', 'shun', 'the', 'frumious', 'bandersnatch', 'he', 'took', 'his', 'vorpal', 'sword', 'in', 'hand', 'long', 'time', 'the', 'manxome', 'foe', 'he', 'sought', 'so', 'rested', 'he', 'by', 'the', 'tumtum', 'tree', 'and', 'stood', 'awhile', 'in', 'thought', 'and', 'as', 'in', 'uffish', 'thought', 'he', 'stood', 'the', 'jabberwock', 'with', 'eyes', 'of', 'flame', 'came', 'whiffling', 'through', 'the', 'tulgey', 'wood', 'and', 'burbled', 'as', 'it', 'came', 'one', 'two', 'one', 'two', 'and', 'through', 'and', 'through', 'the', 'vorpal', 'blade', 'went', 'snicker', 'snack', 'he', 'left', 'it', 'dead', 'and', 'with', 'its', 'head', 'he', 'went', 'galumphing', '

In [24]:
parker_words = clean_words(parker_text.lower())
print parker_words

['my', 'answers', 'are', 'inadequate', 'to', 'those', 'demanding', 'day', 'and', 'date', 'and', 'ever', 'set', 'a', 'tiny', 'shock', 'through', 'strangers', 'asking', 'what', 's', 'o', 'clock', 'whose', 'days', 'are', 'spent', 'in', 'whittling', 'rhyme', 'what', 's', 'time', 'to', 'her', 'or', 'she', 'to', 'time']


For some comparisons we'll also need a list of the unique words in one of the poems.

In [31]:
jabber_uniq = sorted(set(jabber_words))
print jabber_uniq

['all', 'and', 'arms', 'as', 'awhile', 'back', 'bandersnatch', 'beamish', 'beware', 'bird', 'bite', 'blade', 'borogoves', 'boy', 'brillig', 'burbled', 'by', 'callay', 'callooh', 'came', 'catch', 'chortled', 'claws', 'come', 'day', 'dead', 'did', 'eyes', 'flame', 'foe', 'frabjous', 'frumious', 'galumphing', 'gimble', 'gyre', 'hand', 'has', 'he', 'head', 'his', 'in', 'it', 'its', 'jabberwock', 'jaws', 'joy', 'jubjub', 'left', 'long', 'manxome', 'mimsy', 'mome', 'my', 'o', 'of', 'one', 'outgrabe', 'raths', 'rested', 'shun', 'slain', 'slithy', 'snack', 'snicker', 'so', 'son', 'sought', 'stood', 'sword', 'that', 'the', 'thou', 'thought', 'through', 'time', 'to', 'took', 'toves', 'tree', 'tulgey', 'tumtum', 'twas', 'two', 'uffish', 'vorpal', 'wabe', 'went', 'were', 'whiffling', 'with', 'wood']


### Set Cardinality

Cardinality is another way of describing how to count the elements in a set.
Let's use [HyperLogLog](http://en.wikipedia.org/wiki/HyperLogLog) to implement a probabilistic counter. We'll count the total number of words in one of the poems.

The main idea here is that when you have a very large collection of things, counting becomes a problem.
In Python, the `long` integers have unlimited precision, so you can count really large sets as long as you have a lot of memory to use.
What if you don't want to use up your application's memory?
What if you need that memory for other purposes?
For example, imagine needing to compare the number of followers on Twitter, when the comparision is between an average user who has about 200 followers and Lady Gaga who has more than 50 million followers.
Do you really care whether Lady Gaga has 50,000,000 or 50,000,100 followers?
The difference is noise anyway, so why waste lots of application memory?
What if there are many celebrities to count and compare?
Instead, **approximate**.

The *HyperLogLog* algorithm was first described in a [paper by Flajolet](http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf) in 2007.
This example uses a [Python implementation](https://github.com/svpcom/hyperloglog) by Vasily Evseenko.

Note that the code in this example is configured to accept a 1% counting error. We'll compare the *approximated count* with the *actual count* to measure the observed error rate.

In [32]:
import hyperloglog

# accept 1% counting error
hll = hyperloglog.HyperLogLog(0.01)

for word in jabber_words:
  hll.add(word)

print "prob count %d, true count %d" % (len(hll), len(jabber_uniq))
print "observed error rate %0.2f" % (abs(len(hll) - len(jabber_uniq)) / float(len(jabber_uniq)))

prob count 90, true count 91
observed error rate 0.01


Great, those results show how the bounded error rate works as expected.

### Set Membership

Set Membership determines whether a specified element is known to be within a given set.
In other words, once we define the set, we can run *membership queries*.
Let's use a [Bloom filter](http://en.wikipedia.org/wiki/Bloom_filter) for approximate membership queries.

One interesting property of a *BloomFilter* is that *false negatives* are not possible.
So the main idea here is that we can load a set, then test elements to see if they are included in the set.
Some small number of elements that aren't in the set will test positive.

Suppose you have a database of known customers for your web site, but the queries in the database are expensive.
If many people visit your web site, but only a fraction are customers, then you could use a *BloomFilter* of the known customers to test whether you even need to query the database.
That could reduce the cost of queries.

The *BloomFilter* was first described in [a paper](https://dl.acm.org/citation.cfm?doid=362686.362692) by Burton Bloom in 1970.
This example uses a [Python implementation](https://github.com/jaybaird/python-bloomfilter) by Jay Baird.

In this case, we'll load the words in *Daylight Saving*, then test the words in *Jabberwocky*.

In [35]:
from pybloom import BloomFilter

bf = BloomFilter(capacity=1000, error_rate=0.001)

for word in parker_words:
  bf.add(word)

intersect = set([])

for word in jabber_words:
  if word in bf:
    intersect.add(word)

print intersect

set(['and', 'in', 'o', 'to', 'through', 'time', 'my', 'day'])


Eight words in common.
Similarly, this approach can be used in database queries, when one side of `JOIN` is much smaller than the other.

### Set Similarity

Thinking about that example above, what if you simply needed an estimate of the number the words in common between the two poems?
What if you have many sets and need quick comparisons about how similar they are?

For example, suppose you want to test a large number of documents for potential plagiarism?
Comparing the similarity of the documents would provide a quick estimate -- enough to weed out the documents that were clearly not similar to each other.

The *MinHash* algorithm was first described in [a paper](http://gatekeeper.dec.com/ftp/pub/dec/SRC/publications/broder/positano-final-wpnums.pdf) by Andrei Broder in 1997.
This example uses a [Python implementation](https://github.com/ekzhu/datasketch) by Eric Zhu.

Here we'll estimate the similarity between the words in the two poems.

In [44]:
from hashlib import sha1
from datasketch import MinHash

def mh_digest (data):
  m = MinHash(num_perm=512)

  for d in data:
    m.digest(sha1(d.encode('utf8')))

  return m

m1 = mh_digest(set(jabber_words))
m2 = mh_digest(set(parker_words))

print "Jaccard simularity %f" % m1.jaccard(m2), "estimated"

s1 = set(jabber_words)
s2 = set(parker_words)
actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))

print "Jaccard simularity %f" % actual_jaccard, "actual"

Jaccard simularity 0.074219 estimated
Jaccard simularity 0.069565 actual


An estimate of 7.4% similarity versus a measure of 7.0% actual similarity, based on the [Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index).
In any case, we're fairly certain that Dorothy Parker didn't plagiarize Lewis Carroll.

### Frequency Summaries

Frequency Summaries are another way of describing the frequency of events in a data stream.
Think: leaderboards used for online competitions, etc.

The main idea is that we want to measure and compare which events are occurring most frequently, and we probably don't care about the events that occur less often.
The precision of the frequencies isn't particularly imporant.

We'll use [Count-min sketch](http://en.wikipedia.org/wiki/Count%E2%80%93min_sketch) for frequency summaries, to implement a probabilistic [word count](http://en.wikipedia.org/wiki/Word_count) on one of the poems.

The *Count-Min Sketch* algorithm was first described in [a paper](http://dl.acm.org/citation.cfm?id=1073718) by Graham Cormode and S. Muthukrishnan in 2005.
This example uses a [Python implementation](https://github.com/IsaacHaze/countminsketch) by Isaac Sijaranamual.

In [52]:
from collections import Counter
from yacms import CountMinSketch

counts = Counter()

# table size=1000, hash functions=10
cms = CountMinSketch(200, 3)

for word in jabber_words:
  counts[word] += 1
  cms.update(word, 1)

Let's take a look at the counts for common words...

In [50]:
for word in ["the", "and", "he", "that"]:
  print "instances of the word `%s` %d" % (word, cms.estimate(word))

instances of the word `the` 19
instances of the word `and` 14
instances of the word `he` 7
instances of the word `that` 2


One practical example would be in language detection: comparing frequencies for the most commonly occuring words is a simply way to predict the language in which a document was written.

Next, let's look at where the estimates in the *sketch* differed from actual counts:

In [51]:
for e in counts:
  if counts[e] != cms.estimate(e):
    print "missed '%s' counter: %d, sketch: %d" % (e, counts[e], cms.estimate(e))

missed 'galumphing' counter: 1, sketch: 2
missed 'head' counter: 1, sketch: 2
missed 'left' counter: 1, sketch: 2


Bokay, that was way better than writting 50 lines of Java code for a Hadoop application.

### Streaming Quantiles

Imagine that you have a large stream of data.
Perhaps you have an application that must measure that stream continually, where stopping to count simply isn't an option.
Suppose you're monitoring ecommerce transations, trying to detect credit card fraud?
Some measure become important (e.g., average order price).
Approximating metrics on a stream is a great way to build applications that are more robust and operate in real-time.

The *t-Digest* algorithm was first described in [a paper](https://github.com/tdunning/t-digest/blob/master/docs/theory/t-digest-paper/histo.pdf) by Ted Dunning and Otmar Ertl in 2013. 
This example uses a [Python implementation](https://github.com/trademob/t-digest) by Trademob GmbH.

In this case, we'll skip the poems.
Instead let's generate a stream of random numbers and look at its *head*, *median*, and *tail*.

In [53]:
from tdigest import TDigest
import random

td = TDigest()

for x in xrange(0, 1000):
    td.add(random.random(), 1)

for q in [0.05, 0.5, 0.95]:
    print "%f @ %f" % (q, td.quantile(q))

0.050000 @ 0.046682
0.500000 @ 0.501629
0.950000 @ 0.947825


The pseudo-random number generator in Python provides something close to a *uniform distibution*.
In other words, it's flat.

### Conclusions

This is just a start. Even so, not so many programmers can claim that they have written and run code that takes advantage of probabilistic data structures. You have.

Consider how this work contrasts with statistics.
In statistical modeling, one fits a data set to a known *probability distribution*, then makes inferences from the model. For example, 
Generally that co



Twitter catch-phrase is: “Hash, don’t sample”

aggregate different ranges by composing the hashes, instead of repeating full-queries

Definitely check out this O'Reilly webcast, [Probabilistic Data Structures and Breaking Down Big Sequence Data](http://www.oreilly.com/pub/e/1784) by C. Titus Brown, about using approximation algorithms in genomics.

[Add ALL The Things](http://www.infoq.com/presentations/abstract-algebra-analytics) by Avi Bryant @ Stripe.

Algebird

MMDS

BlinkDB

[Probabilistic Data Structures for Web Analytics and Data Mining](http://highlyscalable.wordpress.com/2012/05/01/probabilistic-structures-web-analytics-data-mining/) by Ilya Katsov.